In [1]:
import tensorflow as tf 
from tensorflow import keras
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

In [2]:
data_path = 'data/weibo_senti_100k.csv'
data_df = pd.read_csv(data_path, sep=',', error_bad_lines=False)

reviews, labels = list(), list()
max_length = 0
for index, (label, review) in tqdm(data_df.iterrows(), total=data_df.shape[0], desc='to token :'):
    if len(review) > max_length:
        max_length = len(review)
    reviews.append([r for r in review])
    labels.append(int(label))

to token :: 100%|██████████| 119988/119988 [00:08<00:00, 13653.30it/s]


In [3]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000, char_level=True, oov_token='UNK')
tokenizer.fit_on_texts(reviews)
vocab_size = len(tokenizer.word_index.keys())
X = tokenizer.texts_to_sequences(reviews)
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post')
y = np.array(labels)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(5000).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(5000).batch(64)